In [3]:
import sys
print(sys.path)
sys.path.insert(0,'/home/chenkai/Documents/git_projects/gluon-ts/src')

['/home/chenkai/Documents/git_projects/gluon-ts/examples', '/home/chenkai/Documents/git_projects/gluon-ts', '/opt/anaconda/lib/python37.zip', '/opt/anaconda/lib/python3.7', '/opt/anaconda/lib/python3.7/lib-dynload', '', '/home/chenkai/.local/lib/python3.7/site-packages', '/opt/anaconda/lib/python3.7/site-packages', '/opt/anaconda/lib/python3.7/site-packages/pytorchts-0.1.0-py3.7.egg', '/opt/anaconda/lib/python3.7/site-packages/python_rapidjson-0.9.1-py3.7-linux-x86_64.egg', '/opt/anaconda/lib/python3.7/site-packages/pyodds-1.0.0rc1-py3.7.egg', '/opt/anaconda/lib/python3.7/site-packages/hyperopt-0.2.3-py3.7.egg', '/opt/anaconda/lib/python3.7/site-packages/luminol-0.4-py3.7.egg', '/opt/anaconda/lib/python3.7/site-packages/tensorflow-2.0.0b1-py3.7-linux-x86_64.egg', '/opt/anaconda/lib/python3.7/site-packages/networkx-2.2-py3.7.egg', '/opt/anaconda/lib/python3.7/site-packages/TA_Lib-0.4.19-py3.7-linux-x86_64.egg', '/opt/anaconda/lib/python3.7/site-packages/IPython/extensions', '/home/chenk

In [23]:
from functools import partial

import pandas as pd


from gluonts.dataset.repository.datasets import get_dataset
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.mx.trainer import Trainer
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQRNNEstimator,Seq2SeqEstimator # todo:MQCNNEstimator，gpu会报错

In [7]:
epochs = 5
num_batches_per_epoch = 10
dataset_name = "m4_hourly"

In [8]:
dataset = get_dataset(dataset_name,regenerate=False)

In [24]:
# If you want to use GPU, please set ctx="gpu(0)"　/ ctx="cpu"
estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            ctx="gpu(0)",
            epochs=epochs,
            num_batches_per_epoch=num_batches_per_epoch
        )
    ),
    partial(
        MQRNNEstimator,
        trainer=Trainer(
            ctx="gpu(0)",
            epochs=epochs,
            num_batches_per_epoch=num_batches_per_epoch
        )
    ),
    
    partial(
        Seq2SeqEstimator,
        trainer=Trainer(
            ctx="gpu(0)",
            epochs=epochs,
            num_batches_per_epoch=num_batches_per_epoch
        )
    ),
    
]

In [25]:
results = []
for estimator in estimators:
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq
    )
    predictor = estimator.train(dataset.train)
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    results.append(eval_dict)

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


Running evaluation: 100%|██████████| 414/414 [00:01<00:00, 350.71it/s]


ValidationError: 5 validation errors for Seq2SeqEstimatorModel
cardinality
  field required (type=value_error.missing)
embedding_dimension
  field required (type=value_error.missing)
encoder
  field required (type=value_error.missing)
decoder_mlp_layer
  field required (type=value_error.missing)
decoder_mlp_static_dim
  field required (type=value_error.missing)

In [ ]:
df = pd.DataFrame(results)
sub_df = df[
    [
        "dataset",
        "estimator",
        "MSE",
        "RMSE"
    ]
]
print(sub_df)

In [20]:
import numpy as np

In [21]:
np.sqrt(sub_df['MSE'])

0    17625.574842
1    42642.924131
Name: MSE, dtype: float64